NOTE: This notebook is being used to optimize features and relies on files not included in the repository.

In [1]:
import featurecalc as fc

In [9]:
Q1_fl = '/home/dgiles/Documents/KeplerLCs/filelists/Q1_filelist_completed.txt'
with open(Q1_fl,'r') as file:
    f0 = file.readline()[:-1]

path_to_fits = '/home/dgiles/Documents/KeplerLCs/fitsFiles/Q1fitsfiles/'
testfile = path_to_fits+f0

In [999]:
path_to_fits

'/home/dgiles/Documents/KeplerLCs/fitsFiles/Q1fitsfiles/'

In [997]:
files = fc.fl_files_w_path(Q1_fl,path_to_fits)

TypeError: must be str, not numpy.bytes_

In [21]:
import importlib as imp
imp.reload(fc)
nfile,ndata = fc.features_from_fits(testfile)

In [22]:
%timeit fc.features_from_fits(testfile)

488 ms ± 32.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%prun fc.features_from_fits(testfile)

         279376 function calls (277913 primitive calls) in 0.678 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     6501    0.105    0.000    0.452    0.000 function_base.py:3517(_median)
     6517    0.102    0.000    0.102    0.000 {method 'partition' of 'numpy.ndarray' objects}
     7170    0.082    0.000    0.162    0.000 _methods.py:53(_mean)
     7224    0.051    0.000    0.051    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.026    0.026    0.275    0.275 keplerml.py:331(<listcomp>)
     6517    0.025    0.000    0.486    0.000 function_base.py:3369(_ureduce)
     7167    0.021    0.000    0.184    0.000 fromnumeric.py:2843(mean)
    13034    0.019    0.000    0.029    0.000 numerictypes.py:728(issubdtype)
    26929    0.017    0.000    0.017    0.000 {built-in method numpy.core.multiarray.array}
     6517    0.017    0.000    0.017    0.000 {method 'flatten' of 'numpy.ndarray' objects}
     6581 

In [24]:
nfile = testfile
t, nf, err = fc.read_kepler_curve(nfile)

In [26]:
import numpy as np
from scipy import stats

In [27]:
assert len(t)==len(nf) and len(t)==len(err), "t, nf, err arrays must be equal sizes."

"""
The essentials which probably can't be improved much
"""

longtermtrend = np.polyfit(t, nf, 1)[0] # Feature 1 (Abbr. F1) overall slope
yoff = np.polyfit(t, nf, 1)[1] # Not a feature, y-intercept of linear fit
nf_mean = np.mean(nf)
nf_med = np.median(nf)
stds = np.std(nf) #f6
meanmedrat = nf_mean / nf_med # F2
skews = stats.skew(nf) # F3
varss = np.var(nf) # F4
coeffvar = stds/nf_mean #F5

corrnf = nf - longtermtrend*t - yoff #this removes any linear trend of lc so you can look at just troughs

In [40]:
"""
Outlying flux counts
"""

posthreshold = nf_mean+4*stds
negthreshold = nf_mean-4*stds

numout1s = len(nf[np.abs(nf-nf_mean)>stds])
numposoutliers = len(nf[nf>posthreshold])
numnegoutliers = len(nf[nf<negthreshold])


In [53]:
numoutliers=numposoutliers+numnegoutliers #F10

kurt = stats.kurtosis(nf)

mad = np.median(np.abs(nf-nf_med))

In [43]:
# delta nf/delta t
%timeit slopes1 = (nf[1:]-nf[:-1])/(t[1:]-t[:-1])

%timeit slopes2 =[(nf[j+1]-nf[j])/(t[j+1]-t[j]) for j in range(len(nf)-1)]


14.1 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.61 ms ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
slopes = (nf[1:]-nf[:-1])/(t[1:]-t[:-1])

In [47]:
%timeit (corrnf[1:]-corrnf[:-1])/(t[1:]-t[:-1])
%timeit [(corrnf[j+1]-corrnf[j])/(t[j+1]-t[j]) for j in range (len(corrnf)-1)]

16 µs ± 45.7 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
923 µs ± 31.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [48]:
#corrslopes (corrected slopes) removes the longterm linear trend (if any) and then looks at the slope
corrslopes = (corrnf[1:]-corrnf[:-1])/(t[1:]-t[:-1])
meanslope = np.mean(slopes) #F12

In [56]:
# by looking at where the 99th percentile is instead of just the largest number,
# I think it avoids the extremes which might not be relevant (might be unreliable data)
# Is the miniumum slope the most negative one, or the flattest one? Answer: Most negative
maxslope=np.percentile(slopes,99) #F13
minslope=np.percentile(slopes,1) #F14

In [55]:
# Separating positive slopes and negative slopes
# Should both include the 0 slope? It doesn't matter for calculating the means later on...
pslope = slopes[slopes>=0]
nslope = slopes[slopes<=0]
# Looking at the average (mean) positive and negative slopes

if len(pslope)==0:
    meanpslope=0
else:
    meanpslope=np.mean(pslope) #F15

if len(nslope)==0:
    meannslope=0
else:
    meannslope=np.mean(nslope) #F16

In [57]:
# Quantifying the difference in shape.
if meannslope==0:
    # if meannslope==0 (i.e., if there are no negative slopes), g_asymm is assigned a value of 10
    # This value is chosen such that 
    # a) it is positive (where g_asymm is inherently negative), 
    # b) it is a factor larger than a random signal would produce (roughly equal average of positive and negative slopes -> g_asymm=-1)
    # c) it is not orders of magnitude larger than other data, which would affect outlier analysis
    g_asymm = 10 
else:
    g_asymm=meanpslope / meannslope #F17

# Won't this be skewed by the fact that both pslope and nslope have all the 0's? Eh
if len(nslope)==0:
    rough_g_asymm=10
else:
    rough_g_asymm=len(pslope) / len(nslope) #F18

# meannslope is inherently negative, so this is the difference btw the 2
diff_asymm=meanpslope + meannslope #F19
skewslope = stats.skew(slopes) #F20
absslopes = np.abs(slopes)
meanabsslope = np.mean(absslopes) #F21
varabsslope = np.var(absslopes) #F22
varslope = np.var(slopes) #F23

In [60]:
# secder = Second Derivative
# Reminder for self: the slope is "located" halfway between the flux and time points, 
# the delta t in the denominator accounts for that.
# secder = delta slopes/delta t, delta t = ((t_j-t_(j-1))+(t_(j+1)-t_j))/2
# secder=[(slopes[j]-slopes[j-1])/((t[j+1]-t[j])/2+(t[j]-t[j-1])/2) for j in range(1, len(slopes)-1)]
# after algebraic simplification:
secder = 2*(slopes[1:]-slopes[:-1])/(t[1:-1]-t[:-2])

# abssecder=[abs((slopes[j]-slopes[j-1])/((t[j+1]-t[j])/2+(t[j]-t[j-1])/2)) for j in range (1, len(slopes)-1)]
# simplification:

abssecder=np.abs(np.array(secder))
absmeansecder=np.mean(abssecder) #F24
if len(pslope)==0:
    pslopestds=0
else:
    pslopestds=np.std(pslope)
if len(nslope)==0:
    nslopesstds=0
    stdratio=10 # the ratio for noise-only variation will be ~1, so a single order of magnitude seems sufficient.
else:
    nslopestds=np.std(nslope)
    stdratio=pslopestds/nslopestds

sdstds=np.std(secder)
meanstds=np.mean(secder)

num_psdspikes,num_nsdspikes=0,0

num_pspikes=len(slopes[slopes>=meanpslope+3*pslopestds]) #F25
num_nspikes=len(slopes[slopes<=meannslope-3*nslopestds]) #F26

num_psdspikes = len(secder[secder>=4*sdstds]) #F27
num_nsdspikes = len(secder[secder<=4*sdstds]) #F28

if nslopestds==0:
    stdratio=10
else:
    stdratio = pslopestds / nslopestds #F29

In [122]:
# The ratio of postive slopes with a following postive slope to the total number of points.
pairs = np.where((slopes[1:]>0)&(slopes[:-1]>0))[0] # where positive slopes are followed by another positive slope
pstrend=len(pairs)/len(slopes) #F30

In [110]:
# Checks if the flux crosses the 'zero' line.
zcrossind_alt = np.where(corrnf[:-1]*corrnf[1:]<0)
num_zcross = len(zcrossind) #F31

In [157]:
plusminus = np.where((slopes[1:]<0)&(slopes[:-1]>0))[0]
num_pm = len(plusminus)

In [193]:
# This looks up the local maximums. Adds a peak if it's the largest within 10 points on either side.
# Q: Is there a way to do this and take into account drastically different periodicity scales?
naivemax,nmax_times = [],[]
naivemins,nmin_times = [],[]

for j in range(len(nf)):
    if nf[j] == max(nf[max(j-10,0):min(j+10,len(nf)-1)]):
        naivemax.append(nf[j])
        nmax_times.append(t[j])
    elif nf[j] == min(nf[max(j-10,0):min(j+10,len(nf)-1)]):
        naivemins.append(nf[j])
        nmin_times.append(t[j])
        
naivemax = np.array(naivemax)
nmax_times = np.array(nmax_times)
naivemins = np.array(naivemins)
nmin_times = np.array(nmin_times)
len_nmax=len(naivemax) #F33
len_nmin=len(naivemins) #F34

In [191]:
if len(naivemax)>2:
    mautocorrcoef = np.corrcoef(naivemax[:-1], naivemax[1:])[0][1] #F35
else:
    mautocorrcoef = 0

In [929]:
"""peak to peak slopes"""
#ppslopes = [abs((naivemax[j+1]-naivemax[j])/(nmax_times[j+1]-nmax_times[j])) \
#            for j in range(len(naivemax)-1)]
ppslopes = (naivemax[1:]-naivemax[:-1])/(nmax_times[1:]-nmax_times[:-1])
if len(ppslopes)==0:
    ptpslopes = 0
else:
    ptpslopes=np.mean(ppslopes) #F36

#maxdiff=[nmax_times[j+1]-nmax_times[j] for j in range(len(naivemax)-1)]
maxdiff = nmax_times[1:]-nmax_times[:-1]
if len(maxdiff)==0:
    periodicity=0
    periodicityr=0
    naiveperiod=0
else:
    periodicity=np.std(maxdiff)/np.mean(maxdiff) #F37
    periodicityr=np.sum(abs(maxdiff-np.mean(maxdiff)))/np.mean(maxdiff) #F38
    naiveperiod=np.mean(maxdiff) #F39
if len(naivemax)==0:
    maxvars=0
    maxvarsr=0
else:
    maxvars = np.std(naivemax)/np.mean(naivemax) #F40
    maxvarsr = np.sum(abs(naivemax-np.mean(naivemax)))/np.mean(naivemax) #F41

emin = naivemins[::2] # even indice minimums
omin = naivemins[1::2] # odd indice minimums
meanemin = np.mean(emin)
if len(omin)==0:
    meanomin=0
else:
    meanomin = np.mean(omin)
oeratio = meanomin/meanemin #F42

# amp here is actually amp_2 in revantese
# 2x the amplitude (peak-to-peak really)
amp = np.percentile(nf,99)-np.percentile(nf,1) #F43
normamp = amp / np.mean(nf) #this should prob go, since flux is norm'd #F44

# ratio of points within one fifth of the amplitude to the median to total number of points 
#mbp = len(nf[(nf<=(np.median(nf)+0.1*amp))&(nf>=(np.median(nf)-0.1*amp))]) / len(nf) #F45
mbp = len(nf[(nf<=(nf_med+0.1*amp))&(nf>=(nf_med-0.1*amp))]) / len(nf) #F45
f595 = np.percentile(nf,95)-np.percentile(nf,5)
f1090 =np.percentile(nf,90)-np.percentile(nf,10)
f1782 =np.percentile(nf, 82)-np.percentile(nf, 17)
f2575 =np.percentile(nf, 75)-np.percentile(nf, 25)
f3267 =np.percentile(nf, 67)-np.percentile(nf, 32)
f4060 =np.percentile(nf, 60)-np.percentile(nf, 40)
mid20 =f4060/f595 #F46
mid35 =f3267/f595 #F47
mid50 =f2575/f595 #F48
mid65 =f1782/f595 #F49
mid80 =f1090/f595 #F50 

percentamp = max(np.abs(nf-nf_med)/nf_med) #F51
magratio = (max(nf)-nf_med) / amp #F52

autocorrcoef = np.corrcoef(nf[:-1], nf[1:])[0][1] #F53

sautocorrcoef = np.corrcoef(slopes[:-1], slopes[1:])[0][1] #F54

In [928]:
%timeit arr1 = nf[(nf<=(nf_med+0.1*amp))&(nf>=(nf_med-0.1*amp))]
%timeit arr2 = [nf[j] for j in range(len(nf)) if (nf[j] < (np.median(nf) + 0.1*amp)) & (nf[j] > (np.median(nf)-0.1*amp))]

19.5 µs ± 44.7 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
211 ms ± 24.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [926]:
np.array_equal(arr1,arr2)

True

In [568]:
%%timeit
neighbors = 5
flat = np.zeros_like(slopes)
flat[neighbors:-neighbors+1] = slopes[:-2*neighbors+1]
flat[neighbors-1:-neighbors] -= slopes[2*neighbors-1:]
for i in range(1,neighbors):
    flat[neighbors:-neighbors+1]+=slopes[i:-2*neighbors+i+1]
    flat[neighbors-1:-neighbors]-=slopes[2*neighbors-i-1:-i]
flat = flat/(neighbors)
flat = flat[nmax_inds[(nmax_inds>5)&(nmax_inds<len(slopes)-5)]-1] # all the inds are off by one, I don't know why and I don't care

53.2 µs ± 5.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [569]:
%%timeit
flat1 = [np.mean(slopes[max(0,j-6):min(j-1, len(slopes)-1):1]\
        - slopes[max(0,j):min(j+5, len(slopes)-1):1])\
        for j in nmax_inds if (j>5)&(j<len(slopes)-5)]

1.84 ms ± 3.87 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [626]:
""" 
Deprecated. Replaced by method call in following cells.
# this convoluted crap is about 35x faster than what was here before.

nmax_inds = np.argsort(t)[np.searchsorted(t,nmax_times,sorter=np.argsort(t))]
flat = np.zeros_like(slopes)
flat[neighbors:-neighbors+1] = slopes[:-2*neighbors+1]
flat[neighbors-1:-neighbors] -= slopes[2*neighbors-1:]
for i in range(1,neighbors):
    flat[neighbors:-neighbors+1]+=slopes[i:-2*neighbors+i+1]
    flat[neighbors-1:-neighbors]-=slopes[2*neighbors-i-1:-i]
flat = flat/(neighbors)
flatness = flat[nmax_inds[(nmax_inds>5)&(nmax_inds<len(slopes)-5)]-1] 
# all the inds are off by one, I don't know why. I corrected it with a -1 at the end.
# The mixture of indices is messing with my head and I don't know exactly what needs changed.
# At least all of the mismatches are aligned and results are consistent with previous work.
# Fixed in method call below
"""


In [886]:
"""
# Deprecated
%%timeit
flat = mean_n_neighbors(slopes,nbrs=5,comb_method='+-')
flatness = flat[nmax_inds[(nmax_inds>5)&(nmax_inds<len(slopes)-5)]]
"""

486 µs ± 50.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [884]:
# for reference, the old code (itself as optimized as I can make it):
"""
flatness = [np.mean(slopes[max(0,j-6):min(j-1, len(slopes)-1):1]\
        - slopes[max(0,j):min(j+5, len(slopes)-1):1])\
        for j in nmax_inds if (j>5)&(j<len(slopes)-5)]
"""
# njit beats this handily.
def mean_n_neighbors(nparr,nbrs=5,comb_method='++'):
    average_arr = np.zeros_like(nparr)
    exec("average_arr[neighbors+1:-neighbors+1] {}= nparr[:-2*neighbors]".format(comb_method[0]))
    exec("average_arr[neighbors+1:-neighbors+1] {}= nparr[2*neighbors:]".format(comb_method[1]))
    for i in range(1,neighbors):
        exec("average_arr[neighbors+1:-neighbors+1]{}=nparr[i:-2*neighbors+i]".format(comb_method[0]))
        exec("average_arr[neighbors+1:-neighbors+1]{}=nparr[2*neighbors-i:-i]".format(comb_method[1]))
    average_arr = average_arr/neighbors # right now is just over neighbors to match original definitition, 
                                        # should divde by another factor of 2 for the actual mean...
    return average_arr

In [873]:
@njit
def flat_func(slopes,inds):
    flat1 = np.zeros(len(inds))
    for i,j in enumerate(inds):
        flat1[i] = np.mean(slopes[j-6:j-1])-np.mean(slopes[j:j+5])
    return flat1

In [881]:
%%timeit
nmax_inds_subset = nmax_inds[(nmax_inds>5)&(nmax_inds<len(slopes)-5)]
flatness = flat_func(slopes,nmax_inds_subset)

18.4 µs ± 95.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [397]:
if len(flatness)==0: flatmean=0
else: flatmean = np.mean(flatness) #F55

In [725]:
"""
# Optimized and Deprecated
nmin_inds
tflatness = [np.mean(-slopes[j-6:j-1]+ slopes[j:j+5])\
             for j in nmin_inds if (j>5)&(j<len(slopes-5))] 
"""
"""
tflat = mean_n_neighbors(slopes,nbrs=5,comb_method='-+')
tflatness = tflat[nmin_inds[(nmin_inds>5)&(nmin_inds<len(slopes)-5)]]
"""

In [887]:
@njit
def tflat_func(slopes,inds):
    tflat1 = np.zeros(len(inds))
    for i,j in enumerate(inds):
        tflat1[i] = -np.mean(slopes[j-6:j-1])+np.mean(slopes[j:j+5])
    return tflat1

In [894]:
%%timeit
nmin_inds_subset = nmin_inds[(nmin_inds>5)&(nmin_inds<len(slopes)-5)]
tflatness = tflat_func(slopes,nmin_inds_subset)

14.4 µs ± 331 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [398]:
# tflatness for mins, flatness for maxes
if len(tflatness)==0: tflatmean=0
else: tflatmean = np.mean(tflatness) #F56

# Bookmark

I'm inclined to believe that roundness and troundness can be optimized in the same way that flatness and tflatness were. If so, that'd be greeaaat...

In [819]:
from numba import jit, njit, vectorize

In [750]:
%%timeit
# Original
roundness = [np.mean(secder[max(0,j-6):j:1])\
              +np.mean(secder[j:min(j+6,len(secder)-1):1])\
              for j in range(6,len(secder)-6)\
              if t[j] in nmax_times]

5.79 ms ± 654 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [896]:
%%timeit
# Original made better
roundness1 = [np.mean(secder[j-6:j+6])*2\
              for j in nmax_inds if (j>5)&(j<len(secder)-5)]

1.16 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [897]:
np.array_equal(np.around(roundness,decimals=10),np.around(roundness1,decimals=10))

True

In [844]:
@njit
def roundness_func(secder,inds):
    roundness1 = np.zeros(len(inds))
    for i,j in enumerate(inds):
        roundness1[i] = np.mean(secder[j-6:j+6])*2
    return roundness1

In [848]:
%%timeit
nmax_inds_subset = nmax_inds[(nmax_inds>5)&(nmax_inds<len(secder-5))]
roundness2 = roundness_func(secder,nmax_inds_subset)

19.9 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [846]:
np.array_equal(np.around(roundness,decimals=10),np.around(roundness2,decimals=10))

True

In [898]:
nmax_inds_subset = nmax_inds[(nmax_inds>5)&(nmax_inds<len(secder-5))]
roundness = roundness_func(secder,nmax_inds_subset)
if len(roundness)==0: roundmean=0
else: roundmean = np.mean(roundness) #F57

In [904]:
%%timeit
# old
troundness = [np.mean(secder[max(0,j-6):j])\
              +np.mean(secder[j:min(j+6,len(secder)-1)])\
              for j in range(6,len(secder)-6)\
              if t[j] in nmin_times]

5.76 ms ± 500 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [903]:
%%timeit
nmin_inds_subset = nmin_inds[(nmin_inds>5)&(nmin_inds<len(secder-5))]
troundness1 = roundness_func(secder,nmin_inds_subset)

20.1 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [902]:
np.array_equal(np.around(troundness,decimals=10),np.around(troundness1,decimals=10))

True

In [906]:
nmin_inds_subset = nmin_inds[(nmin_inds>5)&(nmin_inds<len(secder-5))]
troundness = roundness_func(secder,nmin_inds_subset)
if len(troundness)==0:troundmean=0
else:troundmean = np.mean(troundness) #F58

In [ ]:
if troundmean==0 and roundmean==0:roundrat=1
elif troundmean==0:roundrat=10
else:roundrat = roundmean / troundmean #F59

if flatmean==0 and tflatmean==0:flatrat=1
elif tflatmean==0:flatrat=10
else:flatrat = flatmean / tflatmean #F60

ndata = np.array([longtermtrend, meanmedrat, skews, varss, coeffvar, stds, \
         numoutliers, numnegoutliers, numposoutliers, numout1s, kurt, mad, \
         maxslope, minslope, meanpslope, meannslope, g_asymm, rough_g_asymm, \
         diff_asymm, skewslope, varabsslope, varslope, meanabsslope, absmeansecder, \
         num_pspikes, num_nspikes, num_psdspikes, num_nsdspikes,stdratio, pstrend, \
         num_zcross, num_pm, len_nmax, len_nmin, mautocorrcoef, ptpslopes, \
         periodicity, periodicityr, naiveperiod, maxvars, maxvarsr, oeratio, \
         amp, normamp, mbp, mid20, mid35, mid50, \
         mid65, mid80, percentamp, magratio, sautocorrcoef, autocorrcoef, \
         flatmean, tflatmean, roundmean, troundmean, roundrat, flatrat])

In [930]:
import time

In [937]:
import pandas as pd
import pickle
import os

In [946]:
fts = ["longtermtrend", "meanmedrat", "skews", "varss", "coeffvar", "stds", \
       "numoutliers", "numnegoutliers", "numposoutliers", "numout1s", "kurt", "mad", \
       "maxslope", "minslope", "meanpslope", "meannslope", "g_asymm", "rough_g_asymm", \
       "diff_asymm", "skewslope", "varabsslope", "varslope", "meanabsslope", "absmeansecder", \
       "num_pspikes", "num_nspikes", "num_psdspikes", "num_nsdspikes","stdratio", "pstrend", \
       "num_zcross", "num_pm", "len_nmax", "len_nmin", "mautocorrcoef", "ptpslopes", \
       "periodicity", "periodicityr", "naiveperiod", "maxvars", "maxvarsr", "oeratio", \
       "amp", "normamp", "mbp", "mid20", "mid35", "mid50", \
       "mid65", "mid80", "percentamp", "magratio", "sautocorrcoef", "autocorrcoef", \
       "flatmean", "tflatmean", "roundmean", "troundmean", "roundrat", "flatrat"]

df = pd.DataFrame(index=[nfile[nfile.find('kplr'):nfile.find('_llc')]],data=[ndata],columns=fts)

In [959]:
fake_fts = ['ft1','ft2','ft3']
fake_data = [range(3),range(3),range(3),range(3)]
fake_df = pd.DataFrame(data=fake_data,columns=fake_fts)

In [960]:
fake_df

,ft1,ft2,ft3
0,0,1,2
1,0,1,2
2,0,1,2
3,0,1,2


In [964]:
fake_df0 = fake_df.iloc[0]
fake_df1 = fake_df.iloc[1]
fake_df2 = fake_df.iloc[2]
fake_df3 = fake_df.iloc[3]

In [981]:
type(fake_df.iloc[0,0])

numpy.int64

In [965]:
with open('./tmp.p','ab') as f:
    pickle.dump(fake_df0,f)
    pickle.dump(fake_df1,f)
    pickle.dump(fake_df2,f)
    pickle.dump(fake_df3,f)

In [957]:
os.remove('./tmp.p')

In [954]:
df_r = pickle.load(open('./tmp.p','rb'))

In [979]:
fake_df_r = pd.DataFrame()
with open('./tmp.p','rb') as fr:
    try:
        while True:
            fake_df_r = fake_df_r.append(pickle.load(fr))
    except EOFError:
        pass

In [980]:
fake_df_r

,ft1,ft2,ft3
0,0.0,1.0,2.0
1,0.0,1.0,2.0
2,0.0,1.0,2.0
3,0.0,1.0,2.0


In [982]:
type(fake_df_r.iloc[0,0])

numpy.float64

In [977]:
fake_df_r = fake_df_r.append(pickle.load(open('./tmp.p','rb')))

In [978]:
fake_df_r

,ft1,ft2,ft3
0,0.0,1.0,2.0
0,0.0,1.0,2.0
0,0.0,1.0,2.0


In [984]:
df.to_csv(open('tmp_data.csv','w'))
pickle.dump(df,open('tmp_data.p','wb'))

In [988]:
df_r = pd.read_csv(open('tmp_data.csv','r'),index_col=0)
df_rp = pickle.load(open('tmp_data.p','rb'))

In [992]:
pd.DataFrame.equals(df,df_r)

False

In [993]:
pd.DataFrame.equals(df,df_rp)

True

In [ ]:
tmpfile='Q8_tmp_data.p'
df = pd.DataFrame()
with open(tmpfile, 'rb') as fr:
    try:
        while True:
            df = df.append(pickle.load(fr))
    except EOFError:
        pass

In [ ]:
df.describe()

In [1004]:
tmpfile='Q8_tmp_data.p'

In [1005]:
df = pd.DataFrame()

In [1011]:
df = pd.DataFrame()
with open(tmpfile, 'rb') as fr:
    try:
        i=0
        while i<10000:
            df = df.append(pickle.load(fr))
            i+=1
    except EOFError:
        pass

In [1013]:
sys.getsizeof(df)

6303024

In [1014]:
import multiprocessing

In [1015]:
def save_output(i):
    """
    Reads in the finished data file (tmp_data.p by default), sorts it, and saves it to the
    specified output csv. Effectively just sorting a csv and renaming it.
    """
    out_file = 'Q{}_output.p'.format(i)
    tmpfile = 'Q{}_tmp_data.p'.format(i)
    df = pd.DataFrame()
    with open(tmpfile, 'rb') as fr:
        try:
            j=0
            while True:
                j+=1
                if j%1000==0:
                    print("Q{}: {}/??? loaded".format(i,j))
                df = df.append(pickle.load(fr))
        except EOFError:
            pass
    df = df.sort_index()
    pickle.dump(df,open(out_file,'wb'))
    os.remove(tmpfile)
    return

In [1101]:
@njit
def n_mins_maxes(t,nf):
    naivemax,nmax_times,nmax_inds = [],[],[]
    naivemins,nmin_times,nmin_inds = [],[],[]
    for j in range(len(nf)):
        nfj = nf[j]
        if j-10<0:
            jmin=0
        else:
            jmin=j+10
        if j+10>len(nf)-1:
            jmax=len(nf-1)
        else:
            jmax=j+10
            
        max_nf=nf[jmin]
        min_nf=nf[jmin]
        for k in range(jmin,jmax):
            if nf[k] >= max_nf:
                max_nf = nf[k]
            elif nf[k] <= min_nf:
                min_nf = nf[k]
        
        if nf[j]==max_nf:
            naivemax.append(nf[j])
            nmax_times.append(t[j])
            nmax_inds.append(j)
        elif nf[j]==min_nf:
            naivemins.append(nf[j])
            nmin_times.append(t[j])
            nmin_inds.append(j)
            
    naivemax = np.array(naivemax)
    nmax_times = np.array(nmax_times)
    nmax_inds = np.array(nmax_inds)
    naivemins = np.array(naivemins)
    nmin_times = np.array(nmin_times)
    nmin_inds = np.array(nmin_inds)
    
    return naivemax,nmax_times,nmax_inds,naivemins,nmin_times,nmin_inds

In [1102]:
%timeit naivemax,nmax_times,nmax_inds,naivemins,nmin_times,nmin_inds=n_mins_maxes(t.astype(np.float64),nf)

18.9 µs ± 717 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [1106]:
def n_mins_maxes(t,nf):
    naivemax,nmax_times = [],[]
    naivemins,nmin_times = [],[]
    for j in range(len(nf)):
        if nf[j] == max(nf[max(j-10,0):min(j+10,len(nf)-1)]):
            naivemax.append(nf[j])
            nmax_times.append(t[j])
        elif nf[j] == min(nf[max(j-10,0):min(j+10,len(nf)-1)]):
            naivemins.append(nf[j])
            nmin_times.append(t[j])
            
    naivemax = np.array(naivemax)
    nmax_times = np.array(nmax_times)
    nmax_inds = np.argsort(t)[np.searchsorted(t,nmax_times,sorter=np.argsort(t))]
    naivemins = np.array(naivemins)
    nmin_times = np.array(nmin_times)
    nmin_inds = np.argsort(t)[np.searchsorted(t,nmin_times,sorter=np.argsort(t))]
    
    return naivemax,nmax_times,nmax_inds,naivemins,nmin_times,nmin_inds


In [1107]:
%timeit naivemax,nmax_times,nmax_inds,naivemins,nmin_times,nmin_inds=n_mins_maxes(t.astype(np.float64),nf)

12 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [1126]:
def n_mins_maxes(nf):
    naivemax,nmax_times = [],[]
    naivemins,nmin_times = [],[]
    for j in range(len(nf)):
        if nf[j] == max(nf[max(j-10,0):min(j+10,len(nf)-1)]):
            naivemax.append(nf[j])
            nmax_times.append(t[j])
        elif nf[j] == min(nf[max(j-10,0):min(j+10,len(nf)-1)]):
            naivemins.append(nf[j])
            nmin_times.append(t[j])
            
    naivemax = np.array(naivemax)
    nmax_times = np.array(nmax_times)
    nmax_inds = np.argsort(t)[np.searchsorted(t,nmax_times,sorter=np.argsort(t))]
    naivemins = np.array(naivemins)
    nmin_times = np.array(nmin_times)
    nmin_inds = np.argsort(t)[np.searchsorted(t,nmin_times,sorter=np.argsort(t))]
    
    return naivemax,nmax_times,nmax_inds,naivemins,nmin_times,nmin_inds

@njit
def easy_feats(t,nf,err):
    nf_mean = np.mean(nf)
    nf_med = np.median(nf)
    stds = np.std(nf) #f6
    meanmedrat = nf_mean / nf_med # F2
    varss = np.var(nf) # F4
    coeffvar = stds/nf_mean #F5

    posthreshold = nf_mean+4*stds
    negthreshold = nf_mean-4*stds

    numout1s = len(nf[np.abs(nf-nf_mean)>stds])
    numposoutliers = len(nf[nf>posthreshold])
    numnegoutliers = len(nf[nf<negthreshold])

    numoutliers=numposoutliers+numnegoutliers #F10

    mad = np.median(np.abs(nf-nf_med))

    # delta nf/delta t
    slopes = (nf[1:]-nf[:-1])/(t[1:]-t[:-1])
    meanslope = np.mean(slopes) #F12

    # Separating positive slopes and negative slopes
    # Should both include the 0 slope? It doesn't matter for calculating the means later on...
    pslope = slopes[slopes>=0]
    nslope = slopes[slopes<=0]
    # Looking at the average (mean) positive and negative slopes
    if len(pslope)==0:meanpslope=0
    else:meanpslope=np.mean(pslope) #F15

    if len(nslope)==0:meannslope=0
    else:meannslope=np.mean(nslope) #F16

    # Quantifying the difference in shape.
    # if meannslope==0 (i.e., if there are no negative slopes), g_asymm is assigned a value of 10
    # This value is chosen such that 
    # a) it is positive (where g_asymm is inherently negative), 
    # b) it is a factor larger than a random signal would produce (roughly equal average of positive and negative slopes -> g_asymm=-1)
    # c) it is not orders of magnitude larger than other data, which would affect outlier analysis
    if meannslope==0:g_asymm = 10
    else:g_asymm=meanpslope / meannslope #F17

    # Won't this be skewed by the fact that both pslope and nslope have all the 0's? Eh
    if len(nslope)==0:rough_g_asymm=10
    else:rough_g_asymm=len(pslope) / len(nslope) #F18

    # meannslope is inherently negative, so this is the difference btw the 2
    diff_asymm=meanpslope + meannslope #F19
    
    absslopes = np.abs(slopes)
    meanabsslope=np.mean(absslopes) #F21
    varabsslope=np.var(absslopes) #F22
    varslope=np.var(slopes) #F23

    # secder = Second Derivative
    # Reminder for self: the slope is "located" halfway between the flux and time points, 
    # so the delta t in the denominator is accounting for that.
    # secder = delta slopes/delta t, delta t = ((t_j-t_(j-1))+(t_(j+1)-t_j))/2
    # secder=[(slopes[j]-slopes[j-1])/((t[j+1]-t[j])/2+(t[j]-t[j-1])/2) for j in range(1, len(slopes)-1)]
    # after algebraic simplification:
    secder = 2*(slopes[1:]-slopes[:-1])/(t[1:-1]-t[:-2])

    # abssecder=[abs((slopes[j]-slopes[j-1])/((t[j+1]-t[j])/2+(t[j]-t[j-1])/2)) for j in range (1, len(slopes)-1)]
    # simplification:

    abssecder=np.abs(secder)
    absmeansecder=np.mean(abssecder) #F24

    if len(pslope)==0:pslopestds=0
    else:pslopestds=np.std(pslope)

    if len(nslope)==0:
        nslopesstds=0
        stdratio=10
    else:
        nslopestds=np.std(nslope)
        stdratio=pslopestds/nslopestds

    sdstds=np.std(secder)
    meanstds=np.mean(secder)

    num_pspikes=len(slopes[slopes>=meanpslope+3*pslopestds]) #F25
    num_nspikes=len(slopes[slopes<=meannslope-3*nslopestds]) #F26

    # 5/30/18, discovered a typo here. meanslope was missing an 'n', i.e. all data
    # processed prior to this date has num_nspikes defined as meanslope-3*nslopestds
    # which will overestimate the number of negative spikes since meanslope is inherently
    # greater than meannslope.

    num_psdspikes = len(secder[secder>=4*sdstds]) #F27
    num_nsdspikes = len(secder[secder<=4*sdstds]) #F28
    if nslopestds==0:
        stdratio=10
    else:
        stdratio = pslopestds / nslopestds #F29

    # The ratio of postive slopes with a following postive slope to the total number of points.
    pairs = np.where((slopes[1:]>0)&(slopes[:-1]>0))[0] # where positive slopes are followed by another positive slope
    pstrend=len(pairs)/len(slopes) #F30

    # Checks if the flux crosses the 'zero' line.
    zcrossind = np.where(corrnf[:-1]*corrnf[1:]<0)
    num_zcross = len(zcrossind) #F31

    plusminus = np.where((slopes[1:]<0)&(slopes[:-1]>0))[0]
    num_pm = len(plusminus)

    # This looks up the local maximums. Adds a peak if it's the largest within 10 points on either side.
    # Q: Is there a way to do this and take into account drastically different periodicity scales?

    naivemax,nmax_times,nmax_inds = [],[],[]
    naivemins,nmin_times,nmin_inds = [],[],[]
    for j in range(len(nf)):
        nfj = nf[j]
        if j-10<0:
            jmin=0
        else:
            jmin=j+10
        if j+10>len(nf)-1:
            jmax=len(nf-1)
        else:
            jmax=j+10
            
        max_nf=nf[jmin]
        min_nf=nf[jmin]
        for k in range(jmin,jmax):
            if nf[k] >= max_nf:
                max_nf = nf[k]
            elif nf[k] <= min_nf:
                min_nf = nf[k]
        
        if nf[j]==max_nf:
            naivemax.append(nf[j])
            nmax_times.append(t[j])
            nmax_inds.append(j)
        elif nf[j]==min_nf:
            naivemins.append(nf[j])
            nmin_times.append(t[j])
            nmin_inds.append(j)
            
    naivemax = np.array(naivemax)
    nmax_times = np.array(nmax_times)
    nmax_inds = np.array(nmax_inds)
    naivemins = np.array(naivemins)
    nmin_times = np.array(nmin_times)
    nmin_inds = np.array(nmin_inds)
    
    len_nmax=len(naivemax) #F33
    len_nmin=len(naivemins) #F34

    ppslopes = (naivemax[1:]-naivemax[:-1])/(nmax_times[1:]-nmax_times[:-1])

    if len(ppslopes)==0:
        ptpslopes = 0
    else:
        ptpslopes=np.mean(ppslopes) #F36

    maxdiff = nmax_times[1:]-nmax_times[:-1]
    
    
    emin = naivemins[::2] # even indice minimums
    omin = naivemins[1::2] # odd indice minimums
    meanemin = np.mean(emin)
    if len(omin)==0:
        meanomin=0
    else:
        meanomin = np.mean(omin)
    oeratio = meanomin/meanemin #F42

    #measures the slope before and after the maximums
    # reminder: 1 less slope than flux, slopes start after first flux
    # slope[0] is between flux[0] and flux[1]
        # mean of slopes before max will be positive
        # mean of slopes after max will be negative

    nmax_inds_subset = nmax_inds[(nmax_inds>5)&(nmax_inds<len(slopes)-5)]
    flatness = np.zeros(len(nmax_inds_subset))
    for i,j in enumerate(nmax_inds_subset):
        flatness[i] = np.mean(slopes[j-6:j-1])-np.mean(slopes[j:j+5])

    if len(flatness)==0: flatmean=0
    else: flatmean = np.mean(flatness) #F55

    # measures the slope before and after the minimums
    # trying flatness w slopes and nf rather than "corr" vals, despite orig def in RN's program
      # mean of slopes before min will be negative
      # mean of slopes after min will be positive


    nmin_inds_subset = nmin_inds[(nmin_inds>5)&(nmin_inds<len(slopes)-5)]
    tflatness = np.zeros(len(nmin_inds_subset))
    for i,j in enumerate(nmin_inds_subset):
        tflatness[i] = -np.mean(slopes[j-6:j-1])+np.mean(slopes[j:j+5])

    # tflatness for mins, flatness for maxes
    if len(tflatness)==0: tflatmean=0
    else: tflatmean = np.mean(tflatness) #F56

    # reminder: 1 less second derivative than slope (2 less than flux). secder starts after first slope.
    # secder[0] is between slope[0] and slope[1], centered at flux[1]

    nmax_inds_subset = nmax_inds[(nmax_inds>5)&(nmax_inds<len(secder-5))]
    
    roundness = np.zeros(len(nmax_inds_subset))
    for i,j in enumerate(nmax_inds_subset):
        roundness[i] = np.mean(secder[j-6:j+6])*2
        
    if len(roundness)==0: roundmean=0
    else: roundmean = np.mean(roundness) #F57

    nmin_inds_subset = nmin_inds[(nmin_inds>5)&(nmin_inds<len(secder-5))]
    troundness = np.zeros(len(nmin_inds_subset))
    for i,j in enumerate(nmin_inds_subset):
        troundness[i] = np.mean(secder[j-6:j+6])*2

    if len(troundness)==0: troundmean=0
    else: troundmean = np.mean(troundness) #F58

    if troundmean==0 and roundmean==0: roundrat=1
    elif troundmean==0: roundrat=10
    else: roundrat = roundmean / troundmean #F59

    if flatmean==0 and tflatmean==0: flatrat=1
    elif tflatmean==0: flatrat=10
    else: flatrat = flatmean / tflatmean #F60"""
        
    return meanmedrat, skews, varss, coeffvar, stds, numoutliers, numnegoutliers, numposoutliers, numout1s, mad, meanpslope, meannslope, g_asymm, rough_g_asymm, diff_asymm, varabsslope, varslope, meanabsslope, absmeansecder, num_pspikes, num_nspikes, num_psdspikes, num_nsdspikes,stdratio, pstrend, num_zcross, num_pm, len_nmax, len_nmin, ptpslopes, oeratio, flatmean, tflatmean, roundmean, troundmean, roundrat, flatrat

def fancy_feats(t,nf,err):
    # fancy meaning I can't throw these under a jit decorator.
    longtermtrend = np.polyfit(t, nf, 1)[0] # Feature 1 (Abbr. F1) overall slope
    yoff = np.polyfit(t, nf, 1)[1] # Not a feature, y-intercept of linear fit
    skews = stats.skew(nf) # F3
    corrnf = nf - longtermtrend*t - yoff #this removes any linear trend of lc so you can look at just troughs
    
    kurt = stats.kurtosis(nf)
    
    # by looking at where the 99th percentile is instead of just the largest number,
    # I think it avoids the extremes which might not be relevant (might be unreliable data)
    # Is the miniumum slope the most negative one, or the flattest one? Answer: Most negative
    maxslope=np.percentile(slopes,99) #F13
    minslope=np.percentile(slopes,1) #F14

    #corrslopes (corrected slopes) removes the longterm linear trend (if any) and then looks at the slope
    corrslopes = (corrnf[1:]-corrnf[:-1])/(t[1:]-t[:-1])
    skewslope = stats.skew(slopes) #F20

    if len(naivemax)>2:
        mautocorrcoef = np.corrcoef(naivemax[:-1], naivemax[1:])[0][1] #F35
    else:
        mautocorrcoef = 0

    if len(maxdiff)==0:
        periodicity=0
        periodicityr=0
        naiveperiod=0
    else:
        periodicity=np.std(maxdiff)/np.mean(maxdiff) #F37
        periodicityr=np.sum(abs(maxdiff-np.mean(maxdiff)))/np.mean(maxdiff) #F38
        naiveperiod=np.mean(maxdiff) #F39
    if len(naivemax)==0:
        maxvars=0
        maxvarsr=0
    else:
        maxvars = np.std(naivemax)/np.mean(naivemax) #F40
        maxvarsr = np.sum(abs(naivemax-np.mean(naivemax)))/np.mean(naivemax) #F41

    # amp here is actually amp_2 in revantese
    # 2x the amplitude (peak-to-peak really)
    amp = np.percentile(nf,99)-np.percentile(nf,1) #F43
    normamp = amp / nf_mean #this should prob go, since flux is norm'd #F44
    
    autocorrcoef = np.corrcoef(nf[:-1], nf[1:])[0][1] #F54

    sautocorrcoef = np.corrcoef(slopes[:-1], slopes[1:])[0][1] #F55
    # ratio of points within one fifth of the amplitude to the median to total number of points 
    mbp = len(nf[(nf<=(nf_med+0.1*amp))&(nf>=(nf_med-0.1*amp))]) / len(nf) #F45

    f595 = np.percentile(nf,95)-np.percentile(nf,5)
    f1090 =np.percentile(nf,90)-np.percentile(nf,10)
    f1782 =np.percentile(nf, 82)-np.percentile(nf, 17)
    f2575 =np.percentile(nf, 75)-np.percentile(nf, 25)
    f3267 =np.percentile(nf, 67)-np.percentile(nf, 32)
    f4060 =np.percentile(nf, 60)-np.percentile(nf, 40)
    mid20 =f4060/f595 #F46
    mid35 =f3267/f595 #F47
    mid50 =f2575/f595 #F48
    mid65 =f1782/f595 #F49
    mid80 =f1090/f595 #F50 

    
    percentamp = max(np.abs(nf-nf_med)/nf_med) #F51

    magratio = (max(nf)-nf_med) / amp #F52
    
    return longtermtrend, kurt, maxslope, minslope, skewslope, mautocorrcoef,periodicity,periodicityr,naiveperiod,maxvars,maxvarsr,amp,normamp,mbp,mid20,mid35,mid50,mid65,mid80,percentamp,magratio,sautocorrcoef,autocorrcoef
    
def feature_calc(t,nf,err):
    meanmedrat, skews, varss, coeffvar, stds, numoutliers, numnegoutliers, numposoutliers, numout1s, mad, meanpslope, meannslope, g_asymm, rough_g_asymm, diff_asymm, varabsslope, varslope, meanabsslope, absmeansecder, num_pspikes, num_nspikes, num_psdspikes, num_nsdspikes,stdratio, pstrend, num_zcross, num_pm, len_nmax, len_nmin, ptpslopes, oeratio, flatmean, tflatmean, roundmean, troundmean, roundrat, flatrat=easy_feats(t,nf,err)
    longtermtrend, kurt, maxslope, minslope, skewslope, mautocorrcoef,periodicity,periodicityr,naiveperiod,maxvars,maxvarsr,amp,normamp,mbp,mid20,mid35,mid50,mid65,mid80,percentamp,magratio,sautocorrcoef,autocorrcoef=fancy_feats(t,nf,err)
    ndata = [longtermtrend, meanmedrat, skews, varss, coeffvar, stds, \
                 numoutliers, numnegoutliers, numposoutliers, numout1s, kurt, mad, \
                 maxslope, minslope, meanpslope, meannslope, g_asymm, rough_g_asymm, \
                 diff_asymm, skewslope, varabsslope, varslope, meanabsslope, absmeansecder, \
                 num_pspikes, num_nspikes, num_psdspikes, num_nsdspikes,stdratio, pstrend, \
                 num_zcross, num_pm, len_nmax, len_nmin, mautocorrcoef, ptpslopes, \
                 periodicity, periodicityr, naiveperiod, maxvars, maxvarsr, oeratio, \
                 amp, normamp, mbp, mid20, mid35, mid50, \
                 mid65, mid80, percentamp, magratio, sautocorrcoef, autocorrcoef, \
                 flatmean, tflatmean, roundmean, troundmean, roundrat, flatrat]
    return ndata

In [1040]:
@njit
def ffunc(a,b):
    c = np.mean(a)+np.mean(b)
    return c

In [1042]:
ffunc(np.array([1,2,3]),np.array([1,2,3]))

4.0

In [1073]:
%timeit feature_calc(t,nf,err)

1.31 ms ± 72.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [1127]:
%%timeit 
feature_calc(t.astype(np.float64),nf,err)

4.31 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1078]:
%timeit feature_calc.py_func(t.astype(np.float64),nf,err)

460 µs ± 51.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [1128]:
def feature_calc(t,nf,err):
    longtermtrend = np.polyfit(t, nf, 1)[0] # Feature 1 (Abbr. F1) overall slope
    yoff = np.polyfit(t, nf, 1)[1] # Not a feature, y-intercept of linear fit
    nf_mean = np.mean(nf)
    nf_med = np.median(nf)
    stds = np.std(nf) #f6
    meanmedrat = nf_mean / nf_med # F2
    skews = stats.skew(nf) # F3
    varss = np.var(nf) # F4
    coeffvar = stds/nf_mean #F5


    corrnf = nf - longtermtrend*t - yoff #this removes any linear trend of lc so you can look at just troughs

    posthreshold = nf_mean+4*stds
    negthreshold = nf_mean-4*stds

    numout1s = len(nf[np.abs(nf-nf_mean)>stds])
    numposoutliers = len(nf[nf>posthreshold])
    numnegoutliers = len(nf[nf<negthreshold])

    numoutliers=numposoutliers+numnegoutliers #F10
    
    kurt = stats.kurtosis(nf)

    mad = np.median(np.abs(nf-nf_med))

    # delta nf/delta t
    slopes = (nf[1:]-nf[:-1])/(t[1:]-t[:-1])

    #corrslopes (corrected slopes) removes the longterm linear trend (if any) and then looks at the slope
    corrslopes = (corrnf[1:]-corrnf[:-1])/(t[1:]-t[:-1])
    meanslope = np.mean(slopes) #F12

    # by looking at where the 99th percentile is instead of just the largest number,
    # I think it avoids the extremes which might not be relevant (might be unreliable data)
    # Is the miniumum slope the most negative one, or the flattest one? Answer: Most negative
    maxslope=np.percentile(slopes,99) #F13
    minslope=np.percentile(slopes,1) #F14

    # Separating positive slopes and negative slopes
    # Should both include the 0 slope? It doesn't matter for calculating the means later on...
    pslope = slopes[slopes>=0]
    nslope = slopes[slopes<=0]
    # Looking at the average (mean) positive and negative slopes
    if len(pslope)==0:meanpslope=0
    else:meanpslope=np.mean(pslope) #F15

    if len(nslope)==0:meannslope=0
    else:meannslope=np.mean(nslope) #F16

    # Quantifying the difference in shape.
    # if meannslope==0 (i.e., if there are no negative slopes), g_asymm is assigned a value of 10
    # This value is chosen such that 
    # a) it is positive (where g_asymm is inherently negative), 
    # b) it is a factor larger than a random signal would produce (roughly equal average of positive and negative slopes -> g_asymm=-1)
    # c) it is not orders of magnitude larger than other data, which would affect outlier analysis
    if meannslope==0:g_asymm = 10
    else:g_asymm=meanpslope / meannslope #F17

    # Won't this be skewed by the fact that both pslope and nslope have all the 0's? Eh
    if len(nslope)==0:rough_g_asymm=10
    else:rough_g_asymm=len(pslope) / len(nslope) #F18

    # meannslope is inherently negative, so this is the difference btw the 2
    diff_asymm=meanpslope + meannslope #F19
    skewslope = stats.skew(slopes) #F20
    absslopes = np.abs(slopes)
    meanabsslope=np.mean(absslopes) #F21
    varabsslope=np.var(absslopes) #F22
    varslope=np.var(slopes) #F23

    # secder = Second Derivative
    # Reminder for self: the slope is "located" halfway between the flux and time points, 
    # so the delta t in the denominator is accounting for that.
    # secder = delta slopes/delta t, delta t = ((t_j-t_(j-1))+(t_(j+1)-t_j))/2
    # secder=[(slopes[j]-slopes[j-1])/((t[j+1]-t[j])/2+(t[j]-t[j-1])/2) for j in range(1, len(slopes)-1)]
    # after algebraic simplification:
    secder = 2*(slopes[1:]-slopes[:-1])/(t[1:-1]-t[:-2])

    # abssecder=[abs((slopes[j]-slopes[j-1])/((t[j+1]-t[j])/2+(t[j]-t[j-1])/2)) for j in range (1, len(slopes)-1)]
    # simplification:

    abssecder=np.abs(secder)
    absmeansecder=np.mean(abssecder) #F24

    if len(pslope)==0:pslopestds=0
    else:pslopestds=np.std(pslope)

    if len(nslope)==0:
        nslopesstds=0
        stdratio=10
    else:
        nslopestds=np.std(nslope)
        stdratio=pslopestds/nslopestds

    sdstds=np.std(secder)
    meanstds=np.mean(secder)

    num_pspikes=len(slopes[slopes>=meanpslope+3*pslopestds]) #F25
    num_nspikes=len(slopes[slopes<=meannslope-3*nslopestds]) #F26

    # 5/30/18, discovered a typo here. meanslope was missing an 'n', i.e. all data
    # processed prior to this date has num_nspikes defined as meanslope-3*nslopestds
    # which will overestimate the number of negative spikes since meanslope is inherently
    # greater than meannslope.

    num_psdspikes = len(secder[secder>=4*sdstds]) #F27
    num_nsdspikes = len(secder[secder<=4*sdstds]) #F28
    if nslopestds==0:
        stdratio=10
    else:
        stdratio = pslopestds / nslopestds #F29

    # The ratio of postive slopes with a following postive slope to the total number of points.
    pairs = np.where((slopes[1:]>0)&(slopes[:-1]>0))[0] # where positive slopes are followed by another positive slope
    pstrend=len(pairs)/len(slopes) #F30

    # Checks if the flux crosses the 'zero' line.
    zcrossind = np.where(corrnf[:-1]*corrnf[1:]<0)
    num_zcross = len(zcrossind) #F31

    plusminus = np.where((slopes[1:]<0)&(slopes[:-1]>0))[0]
    num_pm = len(plusminus)

    # This looks up the local maximums. Adds a peak if it's the largest within 10 points on either side.
    # Q: Is there a way to do this and take into account drastically different periodicity scales?

    naivemax,nmax_times = [],[]
    naivemins,nmin_times = [],[]
    for j in range(len(nf)):
        if nf[j] == max(nf[max(j-10,0):min(j+10,len(nf)-1)]):
            naivemax.append(nf[j])
            nmax_times.append(t[j])
        elif nf[j] == min(nf[max(j-10,0):min(j+10,len(nf)-1)]):
            naivemins.append(nf[j])
            nmin_times.append(t[j])
    naivemax = np.array(naivemax)
    nmax_times = np.array(nmax_times)
    nmax_inds = np.argsort(t)[np.searchsorted(t,nmax_times,sorter=np.argsort(t))]
    naivemins = np.array(naivemins)
    nmin_times = np.array(nmin_times)
    nmin_inds = np.argsort(t)[np.searchsorted(t,nmin_times,sorter=np.argsort(t))]


    len_nmax=len(naivemax) #F33
    len_nmin=len(naivemins) #F34



    if len(naivemax)>2:
        mautocorrcoef = np.corrcoef(naivemax[:-1], naivemax[1:])[0][1] #F35
    else:
        mautocorrcoef = 0

    """peak to peak slopes"""
    ppslopes = (naivemax[1:]-naivemax[:-1])/(nmax_times[1:]-nmax_times[:-1])

    if len(ppslopes)==0:
        ptpslopes = 0
    else:
        ptpslopes=np.mean(ppslopes) #F36

    maxdiff = nmax_times[1:]-nmax_times[:-1]

    if len(maxdiff)==0:
        periodicity=0
        periodicityr=0
        naiveperiod=0
    else:
        periodicity=np.std(maxdiff)/np.mean(maxdiff) #F37
        periodicityr=np.sum(abs(maxdiff-np.mean(maxdiff)))/np.mean(maxdiff) #F38
        naiveperiod=np.mean(maxdiff) #F39
    if len(naivemax)==0:
        maxvars=0
        maxvarsr=0
    else:
        maxvars = np.std(naivemax)/np.mean(naivemax) #F40
        maxvarsr = np.sum(abs(naivemax-np.mean(naivemax)))/np.mean(naivemax) #F41

    emin = naivemins[::2] # even indice minimums
    omin = naivemins[1::2] # odd indice minimums
    meanemin = np.mean(emin)
    if len(omin)==0:
        meanomin=0
    else:
        meanomin = np.mean(omin)
    oeratio = meanomin/meanemin #F42

    # amp here is actually amp_2 in revantese
    # 2x the amplitude (peak-to-peak really)
    amp = np.percentile(nf,99)-np.percentile(nf,1) #F43
    normamp = amp / nf_mean #this should prob go, since flux is norm'd #F44

    # ratio of points within one fifth of the amplitude to the median to total number of points 
    mbp = len(nf[(nf<=(nf_med+0.1*amp))&(nf>=(nf_med-0.1*amp))]) / len(nf) #F45

    f595 = np.percentile(nf,95)-np.percentile(nf,5)
    f1090 =np.percentile(nf,90)-np.percentile(nf,10)
    f1782 =np.percentile(nf, 82)-np.percentile(nf, 17)
    f2575 =np.percentile(nf, 75)-np.percentile(nf, 25)
    f3267 =np.percentile(nf, 67)-np.percentile(nf, 32)
    f4060 =np.percentile(nf, 60)-np.percentile(nf, 40)
    mid20 =f4060/f595 #F46
    mid35 =f3267/f595 #F47
    mid50 =f2575/f595 #F48
    mid65 =f1782/f595 #F49
    mid80 =f1090/f595 #F50 

    percentamp = max(np.abs(nf-nf_med)/nf_med) #F51

    magratio = (max(nf)-nf_med) / amp #F52

    autocorrcoef = np.corrcoef(nf[:-1], nf[1:])[0][1] #F54

    sautocorrcoef = np.corrcoef(slopes[:-1], slopes[1:])[0][1] #F55

    #measures the slope before and after the maximums
    # reminder: 1 less slope than flux, slopes start after first flux
    # slope[0] is between flux[0] and flux[1]
        # mean of slopes before max will be positive
        # mean of slopes after max will be negative

    nmax_inds_subset = nmax_inds[(nmax_inds>5)&(nmax_inds<len(slopes)-5)]
    flatness = flat_func(slopes,nmax_inds_subset)

    if len(flatness)==0: flatmean=0
    else: flatmean = np.mean(flatness) #F55

    # measures the slope before and after the minimums
    # trying flatness w slopes and nf rather than "corr" vals, despite orig def in RN's program
      # mean of slopes before min will be negative
      # mean of slopes after min will be positive


    nmin_inds_subset = nmin_inds[(nmin_inds>5)&(nmin_inds<len(slopes)-5)]
    tflatness = tflat_func(slopes,nmin_inds_subset)

    # tflatness for mins, flatness for maxes
    if len(tflatness)==0: tflatmean=0
    else: tflatmean = np.mean(tflatness) #F56

    # reminder: 1 less second derivative than slope (2 less than flux). secder starts after first slope.
    # secder[0] is between slope[0] and slope[1], centered at flux[1]

    nmax_inds_subset = nmax_inds[(nmax_inds>5)&(nmax_inds<len(secder-5))]
    roundness = roundness_func(secder,nmax_inds_subset)

    if len(roundness)==0: roundmean=0
    else: roundmean = np.mean(roundness) #F57

    nmin_inds_subset = nmin_inds[(nmin_inds>5)&(nmin_inds<len(secder-5))]
    troundness = roundness_func(secder,nmin_inds_subset)

    if len(troundness)==0: troundmean=0
    else: troundmean = np.mean(troundness) #F58

    if troundmean==0 and roundmean==0: roundrat=1
    elif troundmean==0: roundrat=10
    else: roundrat = roundmean / troundmean #F59

    if flatmean==0 and tflatmean==0: flatrat=1
    elif tflatmean==0: flatrat=10
    else: flatrat = flatmean / tflatmean #F60
        
    ndata = [longtermtrend, meanmedrat, skews, varss, coeffvar, stds, \
                 numoutliers, numnegoutliers, numposoutliers, numout1s, kurt, mad, \
                 maxslope, minslope, meanpslope, meannslope, g_asymm, rough_g_asymm, \
                 diff_asymm, skewslope, varabsslope, varslope, meanabsslope, absmeansecder, \
                 num_pspikes, num_nspikes, num_psdspikes, num_nsdspikes,stdratio, pstrend, \
                 num_zcross, num_pm, len_nmax, len_nmin, mautocorrcoef, ptpslopes, \
                 periodicity, periodicityr, naiveperiod, maxvars, maxvarsr, oeratio, \
                 amp, normamp, mbp, mid20, mid35, mid50, \
                 mid65, mid80, percentamp, magratio, sautocorrcoef, autocorrcoef, \
                 flatmean, tflatmean, roundmean, troundmean, roundrat, flatrat]
    return ndata

In [1129]:
%timeit feature_calc(t,nf,err)

23.3 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
